<a href="https://colab.research.google.com/github/SarangWanodei20/learn_python/blob/master/Practice_Final_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TASK 6: Sales Trend Analysis Using Aggregations

Step-by-Step SQL Solution

1.Monthly Revenue & Order Volume


In [15]:
SELECT
    EXTRACT(YEAR FROM order_date) AS order_year,
    EXTRACT(MONTH FROM order_date) AS order_month,
    SUM(amount) AS monthly_revenue,
    COUNT(DISTINCT order_id) AS total_orders
FROM
    online_sales
GROUP BY
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date)
ORDER BY
    order_year, order_month;


IndentationError: unexpected indent (<ipython-input-15-a3acf1f9a534>, line 2)

The above query shows revenue and order volume for each month sorted by year/month.

2. Limit to a Specific Time Range (e.g., 2023)

In [ ]:
SELECT
    EXTRACT(MONTH FROM order_date) AS order_month,
    SUM(amount) AS monthly_revenue,
    COUNT(DISTINCT order_id) AS total_orders
FROM
    online_sales
WHERE
    EXTRACT(YEAR FROM order_date) = 2023
GROUP BY
    EXTRACT(MONTH FROM order_date)
ORDER BY
    order_month;


How do you group data by month and year in SQL?

By using EXTRACT(YEAR FROM date_col) and EXTRACT(MONTH FROM date_col) in PostgreSQL, or YEAR(date_col) and MONTH(date_col) in MySQL.


What’s the difference between COUNT(*) and COUNT(DISTINCT col)?

COUNT(*): Counts all rows.

COUNT(DISTINCT col): Counts unique values in a column.

How do you calculate monthly revenue?

By using SUM(amount) after grouping data by month and year.

 What are aggregate functions in SQL?

Functions like SUM(), AVG(), MAX(), MIN(), and COUNT() that return a single value from a group of rows.

How to handle NULLs in aggregates?

Aggregates like SUM() and COUNT(col) ignore NULLs.

Use COALESCE() to replace NULLs: SUM(COALESCE(amount, 0)).



What’s the role of ORDER BY and GROUP BY?

GROUP BY: Groups rows to apply aggregation.

ORDER BY: Sorts the result set.



How do you get the top 3 months by sales?

Groups the data by month,

Sums the sales amount to get total revenue per month,

Sorts the results by revenue in descending order, and

Limits the output to the top 3 rows.

In [ ]:
SELECT
    TO_CHAR(order_date, 'YYYY-MM') AS month,
    SUM(amount) AS total_revenue
FROM
    online_sales
GROUP BY
    TO_CHAR(order_date, 'YYYY-MM')
ORDER BY
    total_revenue DESC
LIMIT 3;
